# **Atividade Final – Bioinformática Aplicada à Genômica Médica**

## Etapas do trabalho

**1. Configuração do Ambiente:**

- Preparar o ambiente computacional para rodar o pipeline.
- Verificar a qualidade das sequências fornecidas.

**2. Execução do Pipeline:**

- Detectar variantes germinativas em cada amostra.

- Anotar variantes usando ferramentas apropriadas, como ANNOVAR, VEP, ou similares.

**3. Análise Focada em Genes de Interesse:**

- Filtrar e identificar variantes germinativas em genes relacionados aos fenótipos do cenário clínico.

**4. Discussão e Conclusão:**

Interpretar os resultados em relação aos critérios ACMG.

# Configuração do ambiente

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
%%bash

mkdir /content/drive/MyDrive/AtividadeFinalGerminativo_chr10

In [ ]:
%%bash
MeuDrive="/content/drive/MyDrive/AtividadeFinalGerminativo_chr10"

mkdir $MeuDrive/dados
mkdir $MeuDrive/dados/fastq
mkdir $MeuDrive/dados/bam
mkdir $MeuDrive/dados/vcf
mkdir $MeuDrive/logs
mkdir $MeuDrive/reference
mkdir $MeuDrive/reference/hg38

In [ ]:
%%bash

#!/bin/bash

echo '1 - Instalação de programas'
mkdir -p logs

echo 'Instalando o bwa'
sudo apt install bwa 1>logs/bwa.log 2>logs/bwa.log

echo 'Instalando o fastqc'
sudo apt install fastqc 1>logs/fastqc.log 2>logs/fastqc.log

echo 'Instalando o samtools'
sudo apt install samtools 1>logs/samtools.log 2>logs/samtools.log

echo 'Instalando o bedtools'
sudo apt install bedtools 1>logs/bedtools.log 2>logs/bedtools.log

echo 'Instalando o bgzip'
sudo apt install bgzip 1>logs/bgzip.log 2>logs/bgzip.log

echo 'Instalando o tabix'
sudo apt install tabix 1>logs/tabix.log 2>logs/tabix.log

1 - Instalação de programas
Instalando o bwa
Instalando o fastqc
Instalando o samtools
Instalando o bedtools
Instalando o bgzip
Instalando o tabix


In [ ]:
%%bash

MeuDrive="/content/drive/MyDrive/AtividadeFinalGerminativo_chr10"

echo 'Instalando o gatk'
wget https://github.com/broadinstitute/gatk/releases/download/4.1.8.1/gatk-4.1.8.1.zip 1>$MeuDrive/logs/gatk.log 2>$MeuDrive/logs/gatk.log
unzip gatk-4.1.8.1.zip 1>$MeuDrive/logs/gatk.log 2>$MeuDrive/logs/gatk.log
rm gatk-4.1.8.1.zip

Instalando o gatk


In [ ]:
%%bash
MeuDrive="/content/drive/MyDrive/AtividadeFinalGerminativo_chr10"

echo 'Instalando o picard'
wget https://github.com/broadinstitute/picard/releases/download/2.24.2/picard.jar 1>$MeuDrive/logs/picard.log 2>$MeuDrive/logs/picard.log

echo 'Instalando o snpEff'
wget https://snpeff.blob.core.windows.net/versions/snpEff_latest_core.zip 1>$MeuDrive/logs/snpEff.log 2>$MeuDrive/logs/snpEff.log
unzip snpEff_latest_core.zip 1>$MeuDrive/logs/snpEff.log 2>$MeuDrive/logs/snpEff.log
rm snpEff_latest_core.zip

echo 'Instalando o multiqc'
sudo apt install multiqc 1>$MeuDrive/logs/multiqc.log 2>$MeuDrive/logs/multiqc.log

Instalando o picard
Instalando o snpEff
Instalando o multiqc


In [ ]:
%%bash

MeuDrive="/content/drive/MyDrive/AtividadeFinalGerminativo_chr10"

echo '2 - Preparação do Genoma de Referência'

echo 'Baixando o Genoma de Referência'

mkdir -p reference

# baixando o chr1
curl -s "https://hgdownload.soe.ucsc.edu/goldenPath/hg38/chromosomes/chr10.fa.gz" | \
  gunzip -c > $MeuDrive/reference/hg38.fasta

echo 'Indexando o Genoma de Referência'
bwa index \
  -a bwtsw \
  /content/reference/hg38.fasta 1>$MeuDrive/logs/bwa.log 2>$MeuDrive/logs/bwa.log

samtools faidx $MeuDrive/reference/hg38.fasta

java -jar picard.jar CreateSequenceDictionary \
    REFERENCE=$MeuDrive/reference/hg38.fasta \
    OUTPUT=$MeuDrive//reference/hg38.dict 1>$MeuDrive/logs/picard.log 2>$MeuDrive/logs/picard.log

2 - Preparação do Genoma de Referência
Baixando o Genoma de Referência
Indexando o Genoma de Referência


## Script para Análise Germinativa

Faça o upload dos arquivos necessários e não se esqueça de inicializar as variáveis com as informações da sua amostra.

In [ ]:
%%bash

# !/bin/bash

MeuDrive="/content/drive/MyDrive/AtividadeFinalGerminativo_chr10"

amostra="cap-ngse-b-2019-chr10"

fastq1="$MeuDrive/dados/fastq/cap-ngse-b-2019-chr10_S1_L001_R1_001.fastq.gz"
fastq2="$MeuDrive/dados/fastq/cap-ngse-b-2019-chr10_S1_L001_R2_001.fastq.gz"

echo "Iniciando o processamento da $amostra"

# Criando estrutura de diretórios
mkdir -p $MeuDrive/$amostra/input
mkdir -p $MeuDrive/logs

# Movendo arquivos se ainda não estiverem no destino
if [ -e "$fastq1" ]; then
  echo "Movendo '$fastq1' para o diretório input..."
  mv "$fastq1" "$MeuDrive/$amostra/input/"
else
  echo "O arquivo '$fastq1' não existe."
fi

if [ -e "$fastq2" ]; then
  echo "Movendo '$fastq2' para o diretório input..."
  mv "$fastq2" "$MeuDrive/$amostra/input/"
else
  echo "O arquivo '$fastq2' não existe."
fi

# Definir um cabeçalho para o pipeline
echo "==========================="
echo "Início do Pipeline de Análise de Variantes Germinativas"
echo "==========================="

# Passo 1: Preprocessamento de dados
echo "Passo 1: Preprocessamento de dados (ex. QC, trimming)"

## basename $fastq1 = shortcut do nome do arquivo

input_fastq1="$MeuDrive/$amostra/input/$(basename $fastq1)"
input_fastq2="$MeuDrive/$amostra/input/$(basename $fastq2)"

fastqc "$input_fastq1" >> "$MeuDrive/logs/fastqc.log" 2>&1
fastqc "$input_fastq2" >> "$MeuDrive/logs/fastqc.log" 2>&1

Iniciando o processamento da cap-ngse-b-2019-chr10
Movendo '/content/drive/MyDrive/AtividadeFinalGerminativo_chr10/dados/fastq/cap-ngse-b-2019-chr10_S1_L001_R1_001.fastq.gz' para o diretório input...
Movendo '/content/drive/MyDrive/AtividadeFinalGerminativo_chr10/dados/fastq/cap-ngse-b-2019-chr10_S1_L001_R2_001.fastq.gz' para o diretório input...
Início do Pipeline de Análise de Variantes Germinativas
Passo 1: Preprocessamento de dados (ex. QC, trimming)


In [ ]:
%%bash

MeuDrive="/content/drive/MyDrive/AtividadeFinalGerminativo_chr10"
amostra="cap-ngse-b-2019-chr10"
fastq1="cap-ngse-b-2019-chr10_S1_L001_R1_001.fastq.gz"
fastq2="cap-ngse-b-2019-chr10_S1_L001_R2_001.fastq.gz"
input_fastq1="$MeuDrive/$amostra/input/$fastq1"
input_fastq2="$MeuDrive/$amostra/input/$fastq2"
reference="$MeuDrive/reference/hg38.fasta"

mkdir -p "$MeuDrive/$amostra/output"
mkdir -p "$MeuDrive/logs"

# Verificação se o genoma está indexado
if [ ! -f "${reference}.bwt" ]; then
  echo "Indexando o genoma de referência..."
  bwa index "$reference"
fi

# Validação dos arquivos de entrada
if [ ! -f "$input_fastq1" ] || [ ! -f "$input_fastq2" ]; then
  echo "Erro: Arquivo de entrada FASTQ não encontrado!"
  exit 1
fi

# Alinhamento com BWA
echo "Passo 2: Alinhamento de Sequências (ex. BWA)"
bwa mem -R "@RG\tID:$amostra\tSM:$amostra\tLB:$amostra\tPL:ILLUMINA" \
    "$reference" \
    "$input_fastq1" \
    "$input_fastq2" > "$MeuDrive/$amostra/output/$amostra.sam" 2> "$MeuDrive/logs/bwa.log"

if [ $? -eq 0 ]; then
  echo "Alinhamento concluído com sucesso."
else
  echo "Erro durante o alinhamento. Verifique os logs."
fi


Indexando o genoma de referência...
Passo 2: Alinhamento de Sequências (ex. BWA)
Alinhamento concluído com sucesso.


[bwa_index] Pack FASTA... 1.20 sec
[bwa_index] Construct BWT for the packed sequence...
[BWTIncCreate] textLength=267594844, availableWord=30828588
[BWTIncConstructFromPacked] 10 iterations done. 50853228 characters processed.
[BWTIncConstructFromPacked] 20 iterations done. 93947292 characters processed.
[BWTIncConstructFromPacked] 30 iterations done. 132245372 characters processed.
[BWTIncConstructFromPacked] 40 iterations done. 166280796 characters processed.
[BWTIncConstructFromPacked] 50 iterations done. 196527516 characters processed.
[BWTIncConstructFromPacked] 60 iterations done. 223406844 characters processed.
[BWTIncConstructFromPacked] 70 iterations done. 247293244 characters processed.
[BWTIncConstructFromPacked] 80 iterations done. 267594844 characters processed.
[bwt_gen] Finished constructing BWT in 80 iterations.
[bwa_index] 110.79 seconds elapse.
[bwa_index] Update BWT... 0.91 sec
[bwa_index] Pack forward-only FASTA... 0.81 sec
[bwa_index] Construct SA from BWT and Occ.

In [ ]:
%%bash

MeuDrive="/content/drive/MyDrive/AtividadeFinalGerminativo_chr10"
amostra="cap-ngse-b-2019-chr10"


# Passo 3: Conversão de formato e indexação
echo "Passo 3: Conversão para BAM e indexação"
# Conversão SAM para BAM e indexação
samtools sort -O bam -o $MeuDrive/$amostra/output/$amostra.bam $MeuDrive/$amostra/output/$amostra.sam
samtools index $MeuDrive/$amostra/output/$amostra.bam

Passo 3: Conversão para BAM e indexação


In [ ]:
%%bash
MeuDrive="/content/drive/MyDrive/AtividadeFinalGerminativo_chr10"
amostra="cap-ngse-b-2019-chr10"

bedtools bamtobed -i $MeuDrive/$amostra/output/$amostra.bam >$MeuDrive/$amostra/output/$amostra.bed
bedtools merge -i $MeuDrive/$amostra/output/$amostra.bed >$MeuDrive/$amostra/output/$amostra.merged.bed
bedtools sort -i $MeuDrive/$amostra/output/$amostra.merged.bed >$MeuDrive/$amostra/output/$amostra.sorted.bed

In [ ]:
%%bash

MeuDrive="/content/drive/MyDrive/AtividadeFinalGerminativo_chr10"
amostra="cap-ngse-b-2019-chr10"

java -jar picard.jar MarkDuplicates \
    I=$MeuDrive/$amostra/output/$amostra.bam \
    O=$MeuDrive/$amostra/output/$amostra.marked.bam \
    M=$MeuDrive/$amostra/output/$amostra.metrics.txt \
    REMOVE_DUPLICATES=true

INFO	2025-02-07 14:48:31	MarkDuplicates	

********** NOTE: Picard's command line syntax is changing.
**********
********** For more information, please see:
********** https://github.com/broadinstitute/picard/wiki/Command-Line-Syntax-Transition-For-Users-(Pre-Transition)
**********
********** The command line looks like this in the new syntax:
**********
**********    MarkDuplicates -I /content/drive/MyDrive/AtividadeFinalGerminativo_chr10/cap-ngse-b-2019-chr10/output/cap-ngse-b-2019-chr10.bam -O /content/drive/MyDrive/AtividadeFinalGerminativo_chr10/cap-ngse-b-2019-chr10/output/cap-ngse-b-2019-chr10.marked.bam -M /content/drive/MyDrive/AtividadeFinalGerminativo_chr10/cap-ngse-b-2019-chr10/output/cap-ngse-b-2019-chr10.metrics.txt -REMOVE_DUPLICATES true
**********


14:48:32.889 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/content/picard.jar!/com/intel/gkl/native/libgkl_compression.so
[Fri Feb 07 14:48:32 UTC 2025] MarkDuplicates INPUT=[/content/drive/MyDriv

In [ ]:
%%bash

MeuDrive="/content/drive/MyDrive/AtividadeFinalGerminativo_chr10"
amostra="cap-ngse-b-2019-chr10"

samtools index $MeuDrive/$amostra/output/$amostra.marked.bam

# Passo 4: Chamadas de variantes
echo "Passo 4: Chamadas de variantes (GATK HaplotypeCaller)"
# Chamada de variantes
gatk-4.1.8.1/gatk HaplotypeCaller --verbosity ERROR \
    -R $MeuDrive/reference/hg38.fasta \
    -I $MeuDrive/$amostra/output/$amostra.marked.bam \
    -O $MeuDrive/$amostra/output/$amostra.vcf

bgzip -f $MeuDrive/$amostra/output/$amostra.vcf
tabix -f $MeuDrive/$amostra/output/$amostra.vcf.gz

Passo 4: Chamadas de variantes (GATK HaplotypeCaller)


Using GATK jar /content/gatk-4.1.8.1/gatk-package-4.1.8.1-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /content/gatk-4.1.8.1/gatk-package-4.1.8.1-local.jar HaplotypeCaller --verbosity ERROR -R /content/drive/MyDrive/AtividadeFinalGerminativo_chr10/reference/hg38.fasta -I /content/drive/MyDrive/AtividadeFinalGerminativo_chr10/cap-ngse-b-2019-chr10/output/cap-ngse-b-2019-chr10.marked.bam -O /content/drive/MyDrive/AtividadeFinalGerminativo_chr10/cap-ngse-b-2019-chr10/output/cap-ngse-b-2019-chr10.vcf
[February 7, 2025 at 2:56:10 PM UTC] org.broadinstitute.hellbender.tools.walkers.haplotypecaller.HaplotypeCaller done. Elapsed time: 6.69 minutes.
Runtime.totalMemory()=1160773632


In [ ]:
%%bash

MeuDrive="/content/drive/MyDrive/AtividadeFinalGerminativo_chr10"
amostra="cap-ngse-b-2019-chr10"

# Passo 5: Relatório final
echo "Passo 5: Geração do relatório final"
# Geração de relatórios ou visualizações
multiqc $MeuDrive/$amostra/

##abrir o multiqc_report e verificar tbm quais ferramentas o multiqc conhece e podemos rodar esse comando com reports de qualidade

# Fim do pipeline
echo "==========================="
echo "Pipeline de Bioinformática Concluído!"
echo "==========================="

Passo 5: Geração do relatório final
|         searching | ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 18/18  
Pipeline de Bioinformática Concluído!



  /// MultiQC 🔍 | v1.12

|           multiqc | MultiQC Version v1.27 now available!
|           multiqc | Search path : /content/drive/MyDrive/AtividadeFinalGerminativo_chr10/cap-ngse-b-2019-chr10
/usr/local/lib/python3.11/dist-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "
|            picard | Found 1 MarkDuplicates reports
|            fastqc | Found 2 reports
|           multiqc | Compressing plot data
|           multiqc | Report      : multiqc_report.html
|           multiqc | Data        : multiqc_data
|           multiqc | MultiQC complete


In [ ]:
## PASSO 6 - ANOTACAO DE VARIANTES

%%bash

wget https://snpeff.blob.core.windows.net/versions/snpEff_latest_core.zip
unzip -o snpEff_latest_core.zip
rm snpEff_latest_core.zip

Archive:  snpEff_latest_core.zip
  inflating: snpEff/LICENSE.md       
  inflating: snpEff/snpEff.jar       
  inflating: snpEff/SnpSift.jar      
  inflating: snpEff/galaxy/snpSift_int.xml  
  inflating: snpEff/galaxy/tool-data/snpEff_genomes.loc  
  inflating: snpEff/galaxy/tool-data/snpEff_genomes.loc.sample  
  inflating: snpEff/galaxy/snpEffWrapper.pl  
  inflating: snpEff/galaxy/snpEff.xml  
  inflating: snpEff/galaxy/tool_conf.xml  
  inflating: snpEff/galaxy/snpSift_caseControl.xml  
  inflating: snpEff/galaxy/snpSift_filter.xml  
  inflating: snpEff/galaxy/snpSift_annotate.xml  
  inflating: snpEff/galaxy/snpSiftWrapper.pl  
  inflating: snpEff/galaxy/tool_dependencies.xml  
  inflating: snpEff/galaxy/snpEff_download.xml  
  inflating: snpEff/snpEff.config    
  inflating: snpEff/examples/samples_cancer.txt  
  inflating: snpEff/examples/example_motif.vcf  
  inflating: snpEff/examples/cancer.eff.vcf  
  inflating: snpEff/examples/examples.sh  
  inflating: snpEff/examples/tes

--2025-02-07 14:57:22--  https://snpeff.blob.core.windows.net/versions/snpEff_latest_core.zip
Resolving snpeff.blob.core.windows.net (snpeff.blob.core.windows.net)... 52.239.234.228
Connecting to snpeff.blob.core.windows.net (snpeff.blob.core.windows.net)|52.239.234.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 66475427 (63M) [application/zip]
Saving to: ‘snpEff_latest_core.zip’

     0K .......... .......... .......... .......... ..........  0%  143K 7m34s
    50K .......... .......... .......... .......... ..........  0%  286K 5m40s
   100K .......... .......... .......... .......... ..........  0%  143K 6m17s
   150K .......... .......... .......... .......... ..........  0% 43.4M 4m43s
   200K .......... .......... .......... .......... ..........  0%  287K 4m31s
   250K .......... .......... .......... .......... ..........  0%  287K 4m23s
   300K .......... .......... .......... .......... ..........  0%  159M 3m46s
   350K .......... .......... ..

In [ ]:
%%bash

sudo apt update
sudo apt install openjdk-21-jre

java -jar snpEff/snpEff.jar download -v GRCh38.p14

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,309 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,653 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,606 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpad



W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
00:00:00 SnpEff version SnpEff 5.2e (build 2024-10-04 18:09), by Pablo Cingolani
00:00:00 Command: 'download'
00:00:00 Reading configuration file 'snpEff.config'. Genome: 'GRCh38.p14'
00:00:00 Looking for config file: '/content/snpEff.config'
00:00:00 Reading config file: /content/snpEff/snpEff.config
00:00:01 Codon table 'Vertebrate_Mitochondrial' assigne

In [ ]:
%%bash
MeuDrive="/content/drive/MyDrive/AtividadeFinalGerminativo_chr10"
amostra="cap-ngse-b-2019-chr10"

java -Xmx8g -jar snpEff/snpEff.jar -v GRCh38.p14 \
    -stats $MeuDrive/$amostra/output/$amostra.html \
    $MeuDrive/$amostra/output/$amostra.vcf.gz > $MeuDrive/$amostra/output/$amostra.ann.vcf

00:00:00 SnpEff version SnpEff 5.2e (build 2024-10-04 18:09), by Pablo Cingolani
00:00:00 Command: 'ann'
00:00:00 Reading configuration file 'snpEff.config'. Genome: 'GRCh38.p14'
00:00:00 Looking for config file: '/content/snpEff.config'
00:00:00 Reading config file: /content/snpEff/snpEff.config
00:00:01 Codon table 'Vertebrate_Mitochondrial' assigned to chromosome 'MT'
00:00:01 Codon table 'Vertebrate_Mitochondrial' assigned to chromosome 'M'
00:00:01 done
00:00:01 Reading database for genome version 'GRCh38.p14' from file '/content/snpEff/./data/GRCh38.p14/snpEffectPredictor.bin' (this might take a while)
00:01:00 done
00:01:00 Loading Motifs and PWMs
00:01:00 Building interval forest
00:01:17 done.
00:01:17 Genome stats :
#-----------------------------------------------
# Genome name                : 'Human genome GRCh38 using RefSeq transcripts'
# Genome version             : 'GRCh38.p14'
# Genome ID                  : 'GRCh38.p14[0]'
# Has protein coding info    : true
# Has Tr. 

In [ ]:
%%bash

mkdir snpEff/./db
mkdir snpEff/./db/GRCh38
mkdir snpEff/./db/GRCh38/clinvar
mkdir snpEff/./db/GRCh38/dbSnp


wget -O snpEff/./db/GRCh38/clinvar/clinvar-latest.vcf.gz \
    https://ftp.ncbi.nlm.nih.gov/pub/clinvar/vcf_GRCh38/clinvar.vcf.gz
wget -O snpEff/./db/GRCh38/clinvar/clinvar-latest.vcf.gz.tbi \
    https://ftp.ncbi.nlm.nih.gov/pub/clinvar/vcf_GRCh38/clinvar.vcf.gz.tbi

wget -O snpEff/./db/GRCh38/dbSnp/dbSnp.vcf.gz \
    ftp://ftp.ncbi.nih.gov/snp/organisms/human_9606/VCF/00-common_all.vcf.gz
wget -O snpEff/./db/GRCh38/dbSnp/dbSnp.vcf.gz.tbi \
    ftp://ftp.ncbi.nih.gov/snp/organisms/human_9606/VCF/00-common_all.vcf.gz.tbi

--2025-02-07 15:07:52--  https://ftp.ncbi.nlm.nih.gov/pub/clinvar/vcf_GRCh38/clinvar.vcf.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.12, 130.14.250.10, 130.14.250.11, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.12|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 144145510 (137M) [application/x-gzip]
Saving to: ‘snpEff/./db/GRCh38/clinvar/clinvar-latest.vcf.gz’

     0K .......... .......... .......... .......... ..........  0%  129K 18m7s
    50K .......... .......... .......... .......... ..........  0%  130K 18m6s
   100K .......... .......... .......... .......... ..........  0% 55.1M 12m5s
   150K .......... .......... .......... .......... ..........  0%  259K 11m19s
   200K .......... .......... .......... .......... ..........  0%  183M 9m3s
   250K .......... .......... .......... .......... ..........  0%  215M 7m32s
   300K .......... .......... .......... .......... ..........  0%  246M 6m28s
  

In [ ]:
%%bash
MeuDrive="/content/drive/MyDrive/AtividadeFinalGerminativo_chr10"
amostra="cap-ngse-b-2019-chr10"

java -Xmx1g -jar snpEff/SnpSift.jar \
    annotate \
    snpEff/./db/GRCh38/clinvar/clinvar-latest.vcf.gz \
    $MeuDrive/$amostra/output/$amostra.ann.vcf \
    > $MeuDrive/$amostra/output/$amostra.clinvar.ann.vcf

In [ ]:
%%bash

MeuDrive="/content/drive/MyDrive/AtividadeFinalGerminativo_chr10"
amostra="cap-ngse-b-2019-chr10"

gatk-4.1.8.1/gatk VariantsToTable -V $MeuDrive/$amostra/output/$amostra.clinvar.ann.vcf \
    -F CHROM \
    -F POS \
    -F QUAL \
    -F TYPE \
    -F ID \
    -F ALLELEID \
    -F CLNDN \
    -F CLNSIG \
    -F CLNSIGCONF \
    -F CLNSIGINCL \
    -F CLNVC \
    -F GENEINFO \
    -F AF_EXAC \
    -F CLNHGVS \
    -GF AD \
    -GF DP \
    -GF GQ \
    -GF GT \
    -O $MeuDrive/$amostra/output/$amostra.clinvar.ann.txt

Using GATK jar /content/gatk-4.1.8.1/gatk-package-4.1.8.1-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /content/gatk-4.1.8.1/gatk-package-4.1.8.1-local.jar VariantsToTable -V /content/drive/MyDrive/AtividadeFinalGerminativo_chr10/cap-ngse-b-2019-chr10/output/cap-ngse-b-2019-chr10.clinvar.ann.vcf -F CHROM -F POS -F QUAL -F TYPE -F ID -F ALLELEID -F CLNDN -F CLNSIG -F CLNSIGCONF -F CLNSIGINCL -F CLNVC -F GENEINFO -F AF_EXAC -F CLNHGVS -GF AD -GF DP -GF GQ -GF GT -O /content/drive/MyDrive/AtividadeFinalGerminativo_chr10/cap-ngse-b-2019-chr10/output/cap-ngse-b-2019-chr10.clinvar.ann.txt
15:10:55.691 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/content/gatk-4.1.8.1/gatk-package-4.1.8.1-local.jar!/com/intel/gkl/native/libgkl_compression.so
15:10:56.042 INFO  VariantsToTable - ------------------------------------------------

In [ ]:
# Interpretação de Variantes

MeuDrive = "/content/drive/MyDrive/AtividadeFinalGerminativo_chr10"
amostra = "cap-ngse-b-2019-chr10"

import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np

caminho_arquivo = f"{MeuDrive}/{amostra}/output/{amostra}.clinvar.ann.txt"
df = pd.read_csv(caminho_arquivo, sep="\t")
df

,CHROM,POS,QUAL,TYPE,ID,ALLELEID,CLNDN,CLNSIG,CLNSIGCONF,CLNSIGINCL,CLNVC,GENEINFO,AF_EXAC,CLNHGVS,cap-ngse-b-2019-chr10.AD,cap-ngse-b-2019-chr10.DP,cap-ngse-b-2019-chr10.GQ,cap-ngse-b-2019-chr10.GT
0,chr10,12362,94.64,SNP,.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"4,3",7,99,C/T
1,chr10,12511,179.60,INDEL,.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"33,8",41,99,GAC/G
2,chr10,12539,95.64,SNP,.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"35,9",44,99,G/C
3,chr10,12622,151.64,SNP,.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"9,7",16,99,C/T
4,chr10,17632,37.32,SNP,.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"0,2",2,6,G/G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8768,chr10,133768255,50.64,SNP,.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"3,2",5,58,T/G
8769,chr10,133768281,52.64,SNP,.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"5,2",7,60,A/G
8770,chr10,133768345,50.64,SNP,.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"3,2",5,58,T/G
8771,chr10,133768561,327.64,SNP,.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"12,14",26,99,T/C


In [ ]:
df["TYPE"].value_counts()

,count
TYPE,
SNP,7870
INDEL,900
MIXED,3


In [ ]:
df["QUAL"].describe()

,QUAL
count,8773.000000
mean,372.601945
std,625.252429
min,30.440000
25%,37.320000
50%,113.840000
75%,425.640000
max,8689.060000


In [ ]:
df[["cap-ngse-b-2019-chr10.DP", "QUAL", "cap-ngse-b-2019-chr10.GQ"]].describe()

,cap-ngse-b-2019-chr10.DP,QUAL,cap-ngse-b-2019-chr10.GQ
count,8773.000000,8773.000000,8773.000000
mean,20.508150,372.601945,48.143736
std,31.428236,625.252429,41.668991
min,1.000000,30.440000,1.000000
25%,2.000000,37.320000,6.000000
50%,6.000000,113.840000,32.000000
75%,26.000000,425.640000,99.000000
max,359.000000,8689.060000,99.000000


In [ ]:
df1=df
df1['AF_EXAC'].replace('.','0.0',inplace=True)
df1['AF_EXAC'].value_counts()

<ipython-input-25-938678411721>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df1['AF_EXAC'].replace('.','0.0',inplace=True)


,count
AF_EXAC,
0.61750,2
0.99865,2
0.72291,1
0.37294,1
0.73808,1
...,...
0.89458,1
0.26471,1
0.80309,1


In [ ]:
df['Gene'] = df['INFO'].str.extract(r'ANN=[^|]*\|[^|]*\|([^|]+)')

KeyError: 'INFO'

In [ ]:
patho = df['CLNSIG'].isin(["Pathogenic", "Likely_pathogenic", "Uncertain_significance"])
df[patho]

,CHROM,POS,QUAL,TYPE,ID,ALLELEID,CLNDN,CLNSIG,CLNSIGCONF,CLNSIGINCL,CLNVC,GENEINFO,AF_EXAC,CLNHGVS,cap-ngse-b-2019-chr10.AD,cap-ngse-b-2019-chr10.DP,cap-ngse-b-2019-chr10.GQ,cap-ngse-b-2019-chr10.GT
4489,chr10,70777180,1115.64,SNP,691402,679096.0,Aganglionic_megacolon,Uncertain_significance,NaN,NaN,single_nucleotide_variant,TBATA:219793,0.26886,NC_000010.11:g.70777180A>G,"56,48",104,99,A/G
4674,chr10,72008537,64.64,SNP,289041,273278.0,not_provided|Spondyloepiphyseal_dysplasia_with...,Uncertain_significance,NaN,NaN,single_nucleotide_variant,CHST3:9469,NaN,NC_000010.11:g.72008537G>T,"11,3",14,72,G/T
5076,chr10,80090851,422.64,SNP,3327028,3482676.0,not_specified,Uncertain_significance,NaN,NaN,single_nucleotide_variant,TMEM254:80195,NaN,NC_000010.11:g.80090851C>A,"13,19",32,99,C/A
5166,chr10,82985500,494.64,SNP,691408,679099.0,Aganglionic_megacolon,Uncertain_significance,NaN,NaN,single_nucleotide_variant,NRG3:10718,0.38022,NC_000010.11:g.82985500C>T,"14,18",32,99,C/T
6487,chr10,102402138,844.64,SNP,65385,76319.0,not_specified|Common_variable_immunodeficiency...,Pathogenic,NaN,NaN,single_nucleotide_variant,NFKB2:4791|PSD:5662,NaN,NC_000010.11:g.102402138C>T,"37,36",73,99,C/T
6592,chr10,103139440,341.64,SNP,943906,934943.0,Hereditary_spastic_paraplegia_45|not_provided|...,Uncertain_significance,NaN,NaN,single_nucleotide_variant,NT5C2:22978,0.00030,NC_000010.11:g.103139440C>G,"20,13",33,99,C/G
8301,chr10,131260394,32.64,SNP,2350617,2335967.0,not_specified,Uncertain_significance,NaN,NaN,single_nucleotide_variant,TCERG1L:256536,NaN,NC_000010.11:g.131260394C>T,"1,2",3,32,C/T
